<center><img src="Slide1.png" width = "800"/></center>

### <b> InSAR Coherent Change Detection over Aleppo, Syria (2016)
    
<b><i> Corey Scher<sup>1</sup>
<i>Jamon Van Den Hoek<sup>2</sup>
    
<i><font size=2> 1. Department of Earth and Environmental Sciences | Graduate Center at the City University of New York
    
<i><font size=2> 2. College of Earth, Ocean, and Atmospheric Sciences | Oregon State University

### Introduction

   In this training we will walk through an example of bi-temporal coherent change detection over Aleppo, Syria to map proxies for urban damage that occurred as a result of air bombardments in 2016. We will be using the [Alaska Satellite Facility's](https://asf.alaska.edu/) [Vertex tool](https://search.asf.alaska.edu/#/) to generate two synthetic aperture radar [interferogram](https://appliedsciences.nasa.gov/sites/default/files/Session4-SAR-English_0.pdf) (InSAR) datasets to look for anomalous decreases in InSAR measurements of [coherence](https://en.wikipedia.org/wiki/Coherence_(physics) that occurred following a series of airstrike bombardments. This workflow can be completed either using the ASF [Hyp3 Python SDK](https://hyp3-docs.asf.alaska.edu/), as we will demonstrate in this notebook, or by generating InSAR coherence images manually using the ASF Vertex [user interface](https://search.asf.alaska.edu/#/) and raster band math in a geographic information system interface, such as [QGIS](https://www.qgis.org/en/site/).

In [189]:
import os
import folium
import pandas as pd
import asf_search as asf
from hyp3_sdk import HyP3
import shapely.geometry as shp
import matplotlib.pyplot as plt

In [71]:
#import credentials from .config file
credentials = pd.read_csv('/Users/coreyscher/Documents/GitHub/arset/hyp3.config', header = None, names = ['names', 'values']).set_index('names')

In [75]:
#authenticate to Hyp3 

hyp3 = HyP3(username=credentials.loc['username'][0], 
            password=credentials.loc['password'][0])

In [151]:
wkt = "POLYGON((37.0072 36.091,37.3326 36.091,37.3326 36.3183,37.0072 36.3183,37.0072 36.091))"
results = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1], 
                         intersectsWith=wkt, 
                         maxResults=10,
                        processingLevel = 'SLC').data


In [110]:
def addFootprint(result):
    footprint = shp.Polygon(result.geometry['coordinates'][0])
    gjson = result.geojson()
    fid = gjson['properties']['fileID']
    return fid, footprint

In [153]:
gdf = gpd.GeoDataFrame(list(map(addFootprint, results)), 
                       columns = ['fileID', 'footprint'], 
                       geometry = 'footprint')

In [207]:
from shapely.wkt import dumps, loads

roi = dumps(loads(wkt))

In [226]:

mapa = folium.Map([36.207522, 37.154076],
                zoom_start=7,
                  tiles='cartodbpositron')
gdf.iloc[0:3].explore(m = mapa, 
            column = 'fileID',
           style_kwds = {'opacity': 1, 'fill': False})

In [218]:
gdf.explore?

Signature: gdf.explore(*args, **kwargs)
Docstring:
Interactive map based on folium/leaflet.jsInteractive map based on GeoPandas and folium/leaflet.js

Generate an interactive leaflet map based on :class:`~geopandas.GeoDataFrame`

Parameters
----------
column : str, np.array, pd.Series (default None)
    The name of the dataframe column, :class:`numpy.array`,
    or :class:`pandas.Series` to be plotted. If :class:`numpy.array` or
    :class:`pandas.Series` are used then it must have same length as dataframe.
cmap : str, matplotlib.Colormap, branca.colormap or function (default None)
    The name of a colormap recognized by ``matplotlib``, a list-like of colors,
    :class:`matplotlib.colors.Colormap`, a :class:`branca.colormap.ColorMap` or
    function that returns a named color or hex based on the column
    value, e.g.::

        def my_colormap(value):  # scalar value defined in 'column'
            if value > 1:
                return "green"
            return "red"

color : str, a